# Part 5, Topic 1: ChipWhisperer CPA Attacks in Practice


---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Last time, we went into detail of the various outputs from a CPA attack. This lab gives you a useful starting point for the attack itself.*

**LEARNING OUTCOMES:**

* Build a CPA attack without all the "extra documentation" around it from previous labs.
* Learn about getting data into the expected format.

## Prerequisites

Hold up! Before you continue, check you've done the following tutorials:

* ☑ ChipWhisperer Analyzer CPA Attack (you should have used the analyzer code before).
* ☑ SCA101 Intro (you should have an idea of how to get hardware-specific versions running).

## Connect and Program

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_1_1'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
# If you need to program - run this
fw_path = '../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
# Reset target to kick us off
reset_target(scope)

## Capture

In [ ]:
#Do any special setup here?

In [ ]:
%run "../../Helper_Scripts/plot.ipynb"
plot = real_time_plot(plot_len=3000)

In [ ]:
from tqdm.notebook import trange
ktp = cw.ktp.Basic()

#Set your project name here
project = cw.create_project("projects/lab51_examplecpa", overwrite = True)

#Set your number of traces here
num_traces = 250

for i in trange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
    
    #Send every 10th trace?
    if i % 10 == 0:
        plot.send(trace)
        
project.save()

In [ ]:
#If you don't need scope anymore - run this block
scope.dis()
target.dis()

## Analysis

In [ ]:
import chipwhisperer.analyzer as cwa
#pick right leakage model for your attack
leak_model = cwa.leakage_model.sbox_output
attack = cwa.cpa(project, leak_model)
results = attack.run(cwa.get_jupyter_callback(attack))

### LASCAR Quick Setup

In [ ]:
import chipwhisperer.common.api.lascar as cw_lascar
from lascar import *
cw_container = cw_lascar.CWContainer(project, project.textins)
cpa_engines = [CpaEngine("cpa_%02d" % i, cw_lascar.sbox_HW_gen(i, range(256)) for i in range(16)]
session = Session(cw_container, engines=cpa_engines).run(batch_size=50)

In [ ]:
import chipwhisperer.analyzer as cwa
disp = cw_lascar.LascarDisplay(cpa_engines, list(project.keys[0]))
disp.show_pge()

---
<small>NO-FUN DISCLAIMER: This material is Copyright (C) NewAE Technology Inc., 2015-2020. ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

Tutorials derived from our open-source work must be released under the associated open-source license, and notice of the source must be *clearly displayed*. Only original copyright holders may license or authorize other distribution - while NewAE Technology Inc. holds the copyright for many tutorials, the github repository includes community contributions which we cannot license under special terms and **must** be maintained as an open-source release. Please contact us for special permissions (where possible).

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>